In [ ]:
import re
import requests
from bs4 import BeautifulSoup

base_url = 'https://www.cdw.com/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.5 Safari/605.1.1'
}

record_count = 0
for x in range(1, 2):
    productlist = requests.get(f"https://www.cdw.com/search/software/?w=F&akr=0&ShopAll=true&maxrecords=72&pcurrent={x}", headers=headers)

    print(f"https://www.cdw.com/search/software/?w=F&akr=0&ShopAll=true&maxrecords=72&pcurrent={x}")
    print(productlist.status_code);
    
    if productlist.status_code == 200:
        soup = BeautifulSoup(productlist.content, 'lxml')
        products = soup.find_all('div', class_='search-result coupon-check')
        #product = soup.find_all('div', class_='product-specs col-3')

        for spec in products:
            _name = spec.find('a', 'search-result-product-url').text.strip()
            _manufacturer = re.search("(^[a-zA-Z]*)", _name)
            manufacturer = _manufacturer.group()
            print("Manufacturer: " + manufacturer)
            _partnum = spec.find('span', class_='mfg-code').text.strip()
            partnum = _partnum.strip("MFG#: ")
            print("Part Number: " + partnum)
            newCore = {'manufacturer': manufacturer, 'partNumber': partnum, 'upc': '', 'category': "Software"}
            x = requests.post('http://localhost:8001/api/v1/newCore', json = newCore)
            print(x.text)

            record_count = record_count + 1
        else:
            print("CDW Computer System Pull Complete........")
            print("Total Computer System Records Found: " + str(record_count))

